In [1]:
from pandas import *
import pandas
print pandas.__version__
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime
from geopy.geocoders import Nominatim

%matplotlib inline
matplotlib.style.use('ggplot')




0.19.2


## Load datas


In [2]:
weather = read_csv('201402_babs_open_data/201402_weather_data.csv',index_col=0,parse_dates=True)
status = read_csv('201402_babs_open_data/201402_status_data.csv',index_col=0,parse_dates=True)
stations = read_csv('201402_babs_open_data/201402_station_data.csv',index_col=0,parse_dates=True)

In [55]:
s1 = status[:10]
s2 = weather[:4]
s3 = stations[:4]
s1.loc[:,"station_id"]=s1.index
s1.index=s1.time


In [50]:
# separate date and time to two different columns
def take_time(dt):
    d=datetime.strptime(dt, '%Y/%m/%d %H:%M:%S')
    return d.strftime('%H:%M:%S')
    
def take_date(dt):
    d=datetime.strptime(dt, '%Y/%m/%d %H:%M:%S')
    return d.strftime('%Y-%m-%d')
s1.loc[:,"Date"]=s1.time.apply(take_date)
s1.loc[:,"time"]=s1.time.apply(take_time)
s1.Date=s1.Date.apply(to_datetime)
s1.time=s1.time.apply(to_datetime)


In [0]:
import re

geolocator = Nominatim()

def coor2zip(x):
    loc  = geolocator.reverse(str(x[0])+", "+str(x[1]))
    zips=str(loc.raw['address']['postcode'])
    intzip=re.findall(r"[0-9]{5}", zips)
    return int(intzip[0])

stations.ix[:,'zip']=stations.ix[:,'lat':'long'].apply(coor2zip,axis=1)

In [0]:
# Need internte connection to work!!
# Transfer zip codes to coordinates.
# Instead of apply function to all, it is faster
# to apply to unique values of zip the merge
zcit = Series(weather.zip.unique())

def zip2coor(x):
    loca = geolocator.geocode('CA:'+str(x),timeout=20) 
    # as we are in California added CA:. otherwise work not in all cases
    return Series({'lat':loca.latitude,'long':loca.longitude, 'zip':int(x)})

zipcoor = zcit.apply(zip2coor)
zipcoor.zip=zipcoor.zip.apply(int) #make zip int instead of float.
a=weather.merge(zipcoor, on='zip') #add coordinates to weather

<type 'int'>


<type 'int'>


<type 'int'>


<type 'int'>


<type 'int'>


In [0]:
# the function distance do not use Internet
from geopy.distance import vincenty

coorstat = stations.loc[:,['lat','long']]

def get_cit_coor(x):
    c_stat = (x[0],x[1])
    mdist=1000
    for i in xrange(zipcoor.size):
        c_city = (zipcoor.lat[0],zipcoor.long[0])
        dist = vincenty(c_city, c_stat).kilometers
        if dist < mdist:
            mdist = dist
            coor=c_city
    return Series({'lat':x[0],'long':x[1],'lat_cit':coor[0],'long_cit':coor[1]})

a=stations.ix[:,'lat':'long'].apply(get_cit_coor,axis=1)
stations.merge(a,on=['lat','long'])

In [218]:
stations

                                                     name        lat  \
station_id                                                             
2                       San Jose Diridon Caltrain Station  37.329732   
3                                   San Jose Civic Center  37.330698   
4                                  Santa Clara at Almaden  37.333988   
5                                        Adobe on Almaden  37.331415   
6                                        San Pedro Square  37.336721   
7                                    Paseo de San Antonio  37.333798   
8                                     San Salvador at 1st  37.330165   
9                                               Japantown  37.348742   
10                                     San Jose City Hall  37.337391   
11                                            MLK Library  37.335885   
12                                 SJSU 4th at San Carlos  37.332808   
13                                          St James Park  37.33

In [25]:
from geopy import distance

zcities = weather.zip.unique()
zstations = stations.zip.unique()

def zipdist(zip1,zip2):
    #return distance between two zip codes
    d = distance.distance
    _, wa = geolocator.geocode(zip1,timeout=20)
    _, pa = geolocator.geocode(zip2,timeout=20)
    return (d(wa, pa)).kilometers

def zcit_2_zst(sta,cit):
    # return pandas Dataframe of correspondence between zip codes
    # TODO: can be made faster
    ziptab = {}
    for i in sta:
        mdist = 1e4
        zm = 0
        for j in cit:
            dist = zipdist(i,j)
            if dist < mdist:
                mdist = dist
                zm    = j
        print i,j,mdist,zm
        ziptab[str(i)] = zm
    return DataFrame(data=ziptab,index=['zipc']).transpose()

ziptab = zcit_2_zst(zstations,zcities)

95126 95113 1945.83298315 94301


94104 95113 0.37776864997 94107


94124 95113 7.37773835065 94107


94118 95113 4.87900920235 94107


94103 95113 1.32237174194 94107


...

94063 95113 0.0 94063


95112 95113 0.649942724697 95113


95113 95113 0.0 95113


95128 95113 1947.74837738 94301


95110 95113 0.748799354457 95113


In [0]:
result = s1.join(stations,on='station_id')
res = result.ix[:,['Date','station_id','zip','time','bikes_available','docks_available']]
#result = res.join(weather, on=['Date','zip'])


In [48]:
d = {'col1': 1, 'col2': 2}



        zipc
94040  94041
94041  94041
94063  94063
94102  94107
94103  94107
94104  94107
94105  94107
94107  94107
94108  94107
94109  94107
94111  94107
94118  94107
94124  94107
94133  94107
94301  94301
94306  94041
95110  95113
95112  95113
95113  95113
95126  94301
95128  94301

In [20]:
zstations

array([95110, 95128, 95113, 95112, 94063, 94041, 94040, 94301, 94306,
       94102, 94109, 94111, 94133, 94108, 94105, 94107, 94103, 94118,
       94124, 94104, 95126])

In [10]:
stations.zip.unique()

array([95110, 95128, 95113, 95112, 94063, 94041, 94040, 94301, 94306,
       94102, 94109, 94111, 94133, 94108, 94105, 94107, 94103, 94118,
       94124, 94104, 95126])

In [11]:

d = distance.distance
_, wa = geolocator.geocode('94063')
_, pa = geolocator.geocode('95113')
print((d(wa, pa)).kilometers)


33.3773804556


In [188]:
result = s1.join(s2, on='Date')
result


KeyError: 'Date'

In [190]:
!head -n 10 201402_babs_open_data/201402_trip_data.csv